# Final Models

In this notebook we look at some final candidate models and assess their performance on a validation set

In [1]:
import json
import os
import random
import numpy as np
import pandas as pd

random.seed(17)

### Loading data

We will subsample 50,000 of the 200,000 records for training by sampling 25% of the data from each of the five music categories

In [2]:
data = None
with open(os.path.join('data', 'train.json'), 'r') as train_file:
    data = [json.loads(row) for row in train_file]

In [3]:
data_df = pd.DataFrame(data).drop(columns=['image'])
del data

In [4]:
categories = data_df['category'].unique()
dfs = []
for category in categories:
    dfs.append(data_df[data_df['category'] == category].sample(frac=0.25))
data_df = pd.concat(dfs, axis=0)
data_df = data_df.sort_index()
data_df

,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash
0,4.0,"08 24, 2010",u04428712,"So is Katy Perry's new album ""Teenage Dream"" c...",Amazing that I Actually Bought This...More Ama...,1282608000,Pop,$35.93,p70761125,85559980
8,5.0,"12 4, 2013",u77782870,Kelly sounds great on this CD She is back. I...,GET IT,1386115200,Pop,$6.98,p11658191,15086745
10,3.0,"04 3, 2002",u25030850,Me personally I am not a big fan of Pearl Jam ...,Hmmmmm...........,1017792000,Pop,$6.81,p99659606,41124728
13,5.0,"06 10, 2015",u12668412,very good cd,Five Stars,1433894400,Pop,$2.43,p76494211,71863260
14,5.0,"08 12, 2016",u66989964,She is my fave!,Five Stars,1470960000,Pop,$11.57,p83852395,05580669
...,...,...,...,...,...,...,...,...,...,...
199974,4.0,"01 9, 2007",u17199019,"Listening to this album, there are some great ...",A different approach from JB for sure!,1168300800,Pop,$15.43,p47061408,34190361
199976,4.0,"08 21, 2012",u15729510,His cd has a great sound..there is his talent ...,the great sound of Sanborn,1345507200,Jazz,$22.81,p99893340,66451838
199984,4.0,"11 28, 2015",u10400124,Nearly all his music in one place a very good ...,Excellent Deal,1448668800,Pop,$14.23,p82805261,95444453
199987,4.0,"03 23, 2014",u32710934,"Love priest, halford one of the best live show...",Remaster ?,1395532800,Alternative Rock,$11.89,p75893713,97935046


### Pre-Processing

We apply feature cleaning as prototyped before and then split into a training and validation set, ensuring that the proportion of data points in training vs validation is consistent for each music category

In [5]:
def trim_price(price):
    """Trims `price` to remove the $ sign.
    
    If the price variable does not have the format $x.xx
    then the empty string is returned.
    
    Parameters
    ----------
    price: str
        A string representing a price.
    
    Returns
    -------
    str
        A string representing `price` but with the $ sign removed,
        or the empty string if `price` does not have the correct
        format.
    
    """
    if (not pd.isnull(price) and isinstance(price, str) and
        len(price) > 0 and price[0] == '$'):
        return price[1:]
    return ""

In [6]:
from datetime import datetime

data_df['reviewMonth'] = data_df['reviewTime'].apply(lambda x: x.split(' ')[0])
data_df['reviewYear'] = data_df['reviewTime'].apply(lambda x: x.split(' ')[2])
data_df['reviewHour'] = data_df['unixReviewTime'].apply(lambda x: datetime.fromtimestamp(x).hour)
data_df['reviewMonthYear'] = data_df['reviewYear'] + '-' + data_df['reviewMonth']

data_df['cleanedPrice'] = data_df['price'].apply(lambda x: trim_price(x))
data_df = data_df[data_df['cleanedPrice'] != ""]
data_df['cleanedPrice'] = data_df['cleanedPrice'].astype('float')

data_df['fixedReviewText'] = np.where(pd.isnull(data_df['reviewText']), "", data_df['reviewText'])
data_df['fixedSummary'] = np.where(pd.isnull(data_df['summary']), "", data_df['summary'])
data_df['fullReviewText'] = data_df['fixedSummary'] + " " + data_df['fixedReviewText']

data_df = data_df.drop(columns=['fixedReviewText', 'fixedSummary'])

genres = data_df['category'].unique()

for genre in genres:
    genre_col = "is" + genre.replace(" ", "").replace("&", "")
    data_df[genre_col] = data_df['category'].apply(lambda x: 1 if x == genre else 0)

data_df['reviewWordCount'] = data_df['fullReviewText'].apply(lambda x: len(x.split()))

data_df

<ipython-input-6-d3ba8e2e1b50>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['cleanedPrice'] = data_df['cleanedPrice'].astype('float')
<ipython-input-6-d3ba8e2e1b50>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['fixedReviewText'] = np.where(pd.isnull(data_df['reviewText']), "", data_df['reviewText'])
<ipython-input-6-d3ba8e2e1b50>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,...,reviewHour,reviewMonthYear,cleanedPrice,fullReviewText,isPop,isDanceElectronic,isJazz,isClassical,isAlternativeRock,reviewWordCount
0,4.0,"08 24, 2010",u04428712,"So is Katy Perry's new album ""Teenage Dream"" c...",Amazing that I Actually Bought This...More Ama...,1282608000,Pop,$35.93,p70761125,85559980,...,20,2010-08,35.93,Amazing that I Actually Bought This...More Ama...,1,0,0,0,0,277
8,5.0,"12 4, 2013",u77782870,Kelly sounds great on this CD She is back. I...,GET IT,1386115200,Pop,$6.98,p11658191,15086745,...,19,2013-12,6.98,GET IT Kelly sounds great on this CD She is b...,1,0,0,0,0,34
10,3.0,"04 3, 2002",u25030850,Me personally I am not a big fan of Pearl Jam ...,Hmmmmm...........,1017792000,Pop,$6.81,p99659606,41124728,...,19,2002-04,6.81,Hmmmmm........... Me personally I am not a big...,1,0,0,0,0,77
13,5.0,"06 10, 2015",u12668412,very good cd,Five Stars,1433894400,Pop,$2.43,p76494211,71863260,...,20,2015-06,2.43,Five Stars very good cd,1,0,0,0,0,5
14,5.0,"08 12, 2016",u66989964,She is my fave!,Five Stars,1470960000,Pop,$11.57,p83852395,05580669,...,20,2016-08,11.57,Five Stars She is my fave!,1,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199974,4.0,"01 9, 2007",u17199019,"Listening to this album, there are some great ...",A different approach from JB for sure!,1168300800,Pop,$15.43,p47061408,34190361,...,19,2007-01,15.43,A different approach from JB for sure! Listeni...,1,0,0,0,0,108
199976,4.0,"08 21, 2012",u15729510,His cd has a great sound..there is his talent ...,the great sound of Sanborn,1345507200,Jazz,$22.81,p99893340,66451838,...,20,2012-08,22.81,the great sound of Sanborn His cd has a great ...,0,0,1,0,0,61
199984,4.0,"11 28, 2015",u10400124,Nearly all his music in one place a very good ...,Excellent Deal,1448668800,Pop,$14.23,p82805261,95444453,...,19,2015-11,14.23,Excellent Deal Nearly all his music in one pla...,1,0,0,0,0,31
199987,4.0,"03 23, 2014",u32710934,"Love priest, halford one of the best live show...",Remaster ?,1395532800,Alternative Rock,$11.89,p75893713,97935046,...,20,2014-03,11.89,"Remaster ? Love priest, halford one of the bes...",0,0,0,0,1,22


In [7]:
def calculate_MSE(actuals, predicteds):
    """Calculates the Mean Squared Error between `actuals` and `predicteds`.
    
    Parameters
    ----------
    actuals: np.array
        A numpy array of the actual values.
    predicteds: np.array
        A numpy array of the predicted values.
    
    Returns
    -------
    float
        A float representing the Mean Squared Error between `actuals` and
        `predicteds`.
    
    """
    return (((actuals - predicteds)**2).sum()) / (len(actuals))

In [8]:
from sklearn.model_selection import train_test_split

genres = data_df['category'].unique()
X_train_set = []
X_val_set = []
y_train_set = []
y_val_set = []

for genre in genres:
    genre_df = data_df[data_df['category'] == genre]
    targets = genre_df['overall']
    feature_data = genre_df.drop(columns=['overall'])
    X_train, X_val, y_train, y_val = train_test_split(
        feature_data, targets, shuffle=True, test_size=0.2, random_state=17)
    X_train_set.append(X_train)
    X_val_set.append(X_val)
    y_train_set.append(y_train)
    y_val_set.append(y_val)

X_train = pd.concat(X_train_set)
X_val = pd.concat(X_val_set)
y_train = pd.concat(y_train_set)
y_val = pd.concat(y_val_set)

### Collaborative Filtering

In this model we only need a users ID, the items ID, and 

In [9]:
def user_item_matrix(df, rating_col, user_col, item_col):
    return sp.csr_matrix(df[rating_col], (df[user_col], df[item_col]))

In [10]:
train_data = pd.concat([X_train, pd.DataFrame(y_train, columns=['overall'])], axis=1)
val_data = pd.concat([X_val, pd.DataFrame(y_val, columns=['overall'])], axis=1)

In [11]:
import scipy.sparse as sp

item_matrix = train_data.pivot(index='itemID', columns='reviewerID', values='overall')
item_matrix = item_matrix.fillna(0)
user_item_train_matrix = sp.csr_matrix(item_matrix.values)

In [12]:
global_average = train_data['overall'].mean()

In [13]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5)
model_knn.fit(user_item_train_matrix)
item_neighbors = np.asarray(model_knn.kneighbors(user_item_train_matrix, return_distance=False))

In [14]:
user_matrix = train_data.pivot(index='reviewerID', columns='itemID', values='overall')
user_matrix = user_matrix.fillna(0)
user_item_train_matrix = sp.csr_matrix(user_matrix.values)

In [15]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5)
model_knn.fit(user_item_train_matrix)
user_neighbors = np.asarray(model_knn.kneighbors(user_item_train_matrix, return_distance=False))

In [16]:
train_user_avg = train_data.groupby(train_data['reviewerID'], as_index=False)['overall'].mean()
train_item_avg = train_data.groupby(train_data['itemID'], as_index=False)['overall'].mean()
train_user_avg.columns = ['reviewerID', 'userAverage']
train_item_avg.columns = ['itemID', 'itemAverage']
train_user_avg = train_user_avg.set_index('reviewerID')
train_item_avg = train_item_avg.set_index('itemID')

In [17]:
item_avgs = []
for i in range(len(item_neighbors)):
    item_avgs.append(train_item_avg['itemAverage'][item_matrix.index[item_neighbors[i]]].mean())

item_avgs = pd.concat([pd.DataFrame(item_matrix.index, columns=['itemID']), pd.DataFrame(item_avgs, columns=['itemRating'])], axis=1)

In [18]:
user_avgs = []
for i in range(len(user_neighbors)):
    user_avgs.append(train_user_avg['userAverage'][user_matrix.index[user_neighbors[i]]].mean())

user_avgs = pd.concat([pd.DataFrame(user_matrix.index, columns=['reviewerID']), pd.DataFrame(user_avgs, columns=['userRating'])], axis=1)

In [19]:
def weighted_average_data(X, total_avg, user_avgs, item_avgs):
    """Calculates the error based on the weighted average prediction.
    
    Parameters
    ----------
    X: pd.DataFrame
        The DataFrame of features.
    y: np.array
        A numpy array containing the targets
    total_avg: float
        The average across all users/items.
    user_avgs: pd.DataFrame
        A DataFrame containing the average rating for each user.
    item_avgs: pd.DataFrame
        A DataFrame containing the average rating for each item.
    
    Returns
    -------
    float
        A float representing the mean squared error of the predictions.
    
    """
    df_user = pd.merge(X, user_avgs, how='left', on=['reviewerID'])
    df_final = pd.merge(df_user, item_avgs, how='left', on=['itemID'])
    df_final = df_final[['userRating', 'itemRating']]
    df_final = df_final.fillna(total_avg)
    df_final.index = X.index
    return df_final

In [20]:
X_train_aug = weighted_average_data(X_train, global_average, user_avgs, item_avgs)
X_val_aug = weighted_average_data(X_val, global_average, user_avgs, item_avgs)

In [21]:
X_train_mod = pd.concat([X_train, X_train_aug], axis=1)
X_val_mod = pd.concat([X_val, X_val_aug], axis=1)

In [22]:
def threshold_rating(rating):
    """Thresholds `rating` to lie in the range [1, 5].
    
    Parameters
    ----------
    rating: float
        The rating to be thresholded.
    
    Returns
    -------
    float
        A float representing the thresholded rating.
    
    """
    if rating < 1:
        return 1
    if rating > 5:
        return 5
    return rating

In [23]:
weights = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]

vthreshold = np.vectorize(threshold_rating)

for weight in weights:
    print("Weight: %.2f" % weight)
    print("-------------")
    train_pred = vthreshold((weight * X_train_mod['userRating']) + ((1.0 - weight) * X_train_mod['itemRating']))
    val_pred = vthreshold((weight * X_val_mod['userRating']) + ((1.0 - weight) * X_val_mod['itemRating']))
    print("Training MSE: {}".format(calculate_MSE(y_train, train_pred)))
    print("Validation MSE: {}".format(calculate_MSE(y_val, val_pred)))
    print()

Weight: 0.00
-------------
Training MSE: 0.9063375958329644
Validation MSE: 1.0265327709287875

Weight: 0.10
-------------
Training MSE: 0.8811929772416747
Validation MSE: 1.0043193601179092

Weight: 0.30
-------------
Training MSE: 0.8433587912087248
Validation MSE: 0.9725975039754755

Weight: 0.50
-------------
Training MSE: 0.8221313400419478
Validation MSE: 0.9578156018054741

Weight: 0.70
-------------
Training MSE: 0.8175106237413438
Validation MSE: 0.9599736536079039

Weight: 0.90
-------------
Training MSE: 0.8294966423069127
Validation MSE: 0.9790716593827655

Weight: 1.00
-------------
Training MSE: 0.841717177164512
Validation MSE: 0.9949731450098583



### Language Models

The first languge model is built with TfidfVectorizer (Term Frequency - Inverse Document Frequency)

In [24]:
columns_to_keep = ['cleanedPrice', 'isPop', 'isAlternativeRock', 'isJazz', 'isClassical', 'isDanceElectronic', 'reviewWordCount']
X_train_reg1 = X_train[columns_to_keep]
X_val_reg1 = X_val[columns_to_keep]

In [25]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
X_train_reg1['reviewWordCount'] = X_train_reg1['reviewWordCount'].apply(lambda x: np.log(x))
X_val_reg1['reviewWordCount'] = X_val_reg1['reviewWordCount'].apply(lambda x: np.log(x))

<ipython-input-25-2b5d7d7878ed>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_reg1['reviewWordCount'] = X_train_reg1['reviewWordCount'].apply(lambda x: np.log(x))
<ipython-input-25-2b5d7d7878ed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_reg1['reviewWordCount'] = X_val_reg1['reviewWordCount'].apply(lambda x: np.log(x))


In [47]:
def clean_dataset(df):
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [26]:
X_train_reg1 = clean_dataset(X_train_reg1)
y_train1 = y_train[y_train.index.isin(X_train_reg1.index)]
X_train1 = X_train[X_train.index.isin(X_train_reg1.index)]

X_val_reg1 = clean_dataset(X_val_reg1)
y_val1 = y_val[y_val.index.isin(X_val_reg1.index)]
X_val1 = X_val[X_val.index.isin(X_val_reg1.index)]

<ipython-input-26-550e8da2fe0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-26-550e8da2fe0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [27]:
X_train_mod = X_train_mod[X_train_mod.index.isin(X_train_reg1.index)]
X_val_mod = X_val_mod[X_val_mod.index.isin(X_val_reg1.index)]

In [28]:
X_train_reg1 = min_max_scaler.fit_transform(X_train_reg1)
X_val_reg1 = min_max_scaler.transform(X_val_reg1)

In [103]:
from sklearn.linear_model import Ridge

vthreshold_rating = np.vectorize(threshold_rating)

alphas = [0.0, 0.01, 0.03, 0.1, 0.3]
for alpha in alphas:
    print("Alpha = {}".format(alpha))
    print("------------")
    reg_model = Ridge(alpha=alpha)
    reg_model.fit(X_train_reg1, y_train1)
    print("Training Error: {}".format(calculate_MSE(y_train1, vthreshold_rating(reg_model.predict(X_train_reg1)))))
    print("Validation Error: {}".format(calculate_MSE(y_val1, vthreshold_rating(reg_model.predict(X_val_reg1)))))
    print()

Alpha = 0.0
------------
Training Error: 0.9588535445992827
Validation Error: 0.9262869599431929

Alpha = 0.01
------------
Training Error: 0.9588428957078677
Validation Error: 0.9262350113636552

Alpha = 0.03
------------
Training Error: 0.9588433477027621
Validation Error: 0.9262349757449848

Alpha = 0.1
------------
Training Error: 0.9588449304068484
Validation Error: 0.926234860637202

Alpha = 0.3
------------
Training Error: 0.9588494564503761
Validation Error: 0.9262346102237373



In [91]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def process_review_text(review_text, exclude_text, ps):
    """Pre-processes the text given by `review_text`.
    
    Parameters
    ----------
    review_text: str
        The review text to be processed.
    exclude_text: collection
        A collection of words to be excluded.
    ps: PorterStemmer
        The PorterStemmer used to perform word stemming.
    
    Returns
    -------
    str
        A string representing the processed version of `review_text`.
    
    """
    review = re.sub('[^a-zA-Z0-9]', ' ', review_text).lower().split()
    review = [ps.stem(word) for word in review if not word in exclude_text]
    return ' '.join(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Matthew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
exclude_english = set(stopwords.words('english'))
ps = PorterStemmer()
X_train1['processedReview'] = X_train1['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))
X_val1['processedReview'] = X_val1['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))

<ipython-input-30-0d22f415ca83>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train1['processedReview'] = X_train1['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))


##### TFIDF Model

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer()
X_train_cv1 = cv.fit_transform(X_train1['processedReview'])
X_val_cv1 = cv.transform(X_val1['processedReview'])

In [32]:
import scipy.sparse as sp

X_train_reg1_sp = sp.csr_matrix(X_train_reg1)
X_train_cv_reg1 = sp.hstack((X_train_cv1, X_train_reg1_sp), format='csr')

X_val_reg1_sp = sp.csr_matrix(X_val_reg1)
X_val_cv_reg1 = sp.hstack((X_val_cv1, X_val_reg1_sp), format='csr')

In [35]:
from xgboost import XGBRegressor

learning_rates = [0.03, 0.1, 0.3, 0.5]
estimators = [50, 100, 200, 500, 1000, 2000]
depths = [1, 2, 5]

for learning_rate in learning_rates:
    for estimator in estimators:
        for depth in depths:
            print("Learning Rate: {0}, # Estimators: {1}, Depth: {2}".format(learning_rate, estimator, depth))
            print("--------------------------------------------------")
            xg_reg = XGBRegressor(
                learning_rate=learning_rate, max_depth=depth, n_estimators=estimator)
            xg_reg.fit(X_train_cv_reg1, y_train1)
            print("Training Error: {}".format(calculate_MSE(y_train1, vthreshold_rating(xg_reg.predict(X_train_cv_reg1)))))
            print("Validation Error: {}".format(calculate_MSE(y_val1, vthreshold_rating(xg_reg.predict(X_val_cv_reg1)))))
            print()

Learning Rate: 0.03, # Estimators: 50, Depth: 1
--------------------------------------------------
Training Error: 1.657059470470493
Validation Error: 1.643835435274604

Learning Rate: 0.03, # Estimators: 50, Depth: 2
--------------------------------------------------
Training Error: 1.6152387609816117
Validation Error: 1.6027478863791698

Learning Rate: 0.03, # Estimators: 50, Depth: 5
--------------------------------------------------
Training Error: 1.5064334150057164
Validation Error: 1.5156320233080705

Learning Rate: 0.03, # Estimators: 100, Depth: 1
--------------------------------------------------
Training Error: 0.9278305199606406
Validation Error: 0.9140717888445251

Learning Rate: 0.03, # Estimators: 100, Depth: 2
--------------------------------------------------
Training Error: 0.8659974631467421
Validation Error: 0.8564736279922335

Learning Rate: 0.03, # Estimators: 100, Depth: 5
--------------------------------------------------
Training Error: 0.7210557554945155
Valid

Validation Error: 0.5755454593030278

Learning Rate: 0.3, # Estimators: 1000, Depth: 1
--------------------------------------------------
Training Error: 0.47510323441389246
Validation Error: 0.5700952445865527

Learning Rate: 0.3, # Estimators: 1000, Depth: 2
--------------------------------------------------
Training Error: 0.34608801412853196
Validation Error: 0.5653763316636292

Learning Rate: 0.3, # Estimators: 1000, Depth: 5
--------------------------------------------------
Training Error: 0.12860696671980432
Validation Error: 0.5820806629028914

Learning Rate: 0.3, # Estimators: 2000, Depth: 1
--------------------------------------------------
Training Error: 0.4057182092140655
Validation Error: 0.5615677713800927

Learning Rate: 0.3, # Estimators: 2000, Depth: 2
--------------------------------------------------
Training Error: 0.2572209693941599
Validation Error: 0.5717024770539526

Learning Rate: 0.3, # Estimators: 2000, Depth: 5
---------------------------------------------

It seems that `learning_rate=0.1`, `n_estimators=2000`, and `max_depth=1` provides a very good model

In [36]:
from xgboost import XGBRegressor
xg_reg = XGBRegressor(learning_rate=0.1, n_estimators=2000, max_depth=1)
xg_reg.fit(X_train_cv_reg1, y_train1)

train_MSE = calculate_MSE(y_train1, vthreshold_rating(xg_reg.predict(X_train_cv_reg1)))
val_MSE = calculate_MSE(y_val1, vthreshold_rating(xg_reg.predict(X_val_cv_reg1)))

print("Training error based on XGBoost CountVectorizer prediction: %.3f" % train_MSE)
print("Validation error based on XGBoost CountVectorizer prediction: %.3f" % val_MSE)

Training error based on XGBoost CountVectorizer prediction: 0.517
Validation error based on XGBoost CountVectorizer prediction: 0.579


### Word2Vec

In [79]:
from gensim.models import Word2Vec
w2v = Word2Vec(list(X_train1['processedReview']), size=350, window=10, min_count=1)

In [82]:
def create_review_vector(review_text):
    """Creates a vector for the review given by `review_text`.
    
    The word vectors for each word in the review are averaged
    to build a vector for the review.
    
    Parameters
    ----------
    review_text: str
        The review for which the vector is generated.
    
    Returns
    -------
    vector
        A vector for the review.
    
    """
    review = [word for word in review_text if word in w2v.wv.vocab]
    if len(review) > 0:
        return np.mean(w2v[review], axis=0)
    return np.zeros(350)

In [83]:
X_train1['review_vector'] = X_train1['processedReview'].apply(lambda x: create_review_vector(x))
X_val1['review_vector'] = X_val1['processedReview'].apply(lambda x: create_review_vector(x))

<ipython-input-82-07f028c5b26d>:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  return np.mean(w2v[review], axis=0)
<ipython-input-83-ba0c28ab83e8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train1['review_vector'] = X_train1['processedReview'].apply(lambda x: create_review_vector(x))


In [84]:
def create_review_vec_df(review_vecs, indices):
    """Creates a dataframe from `review_vecs`.
    
    Each numpy array in review_vecs is converted to a 
    column in the resulting dataframe.
    
    Parameters
    ----------
    review_vecs: list
        A list of numpy arrays where each array corresponds
        to the review vector for a review.
    indicies: np.array
        A numpy array of indices for the DataFrame
    
    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from converting `review_vecs`
        to a dataframe.
    
    """
    review_vec_df = pd.DataFrame(np.vstack(review_vecs))
    review_vec_df.columns = ["word" + str(col) for col in review_vec_df.columns]
    review_vec_df.index = indices
    return review_vec_df

In [85]:
X_train_wv = create_review_vec_df(X_train1['review_vector'], X_train1.index)
X_val_wv = create_review_vec_df(X_val1['review_vector'], X_val1.index)

In [86]:
X_train_reg1_df = pd.DataFrame(np.vstack(X_train_reg1))
X_train_reg1_df.index = X_train1.index

X_val_reg1_df = pd.DataFrame(np.vstack(X_val_reg1))
X_val_reg1_df.index = X_val1.index

In [87]:
X_train_wv_reg = pd.concat([X_train_wv, X_train_reg1_df], axis=1)
X_val_wv_reg = pd.concat([X_val_wv, X_val_reg1_df], axis=1)

In [88]:
from sklearn.linear_model import Ridge

alphas = [0.0, 0.01, 0.03, 0.1, 0.3]
for alpha in alphas:
    print("Alpha = {}".format(alpha))
    print("------------")
    reg_model = Ridge(alpha=alpha)
    reg_model.fit(X_train_wv_reg, y_train1)
    print("Training Error: {}".format(calculate_MSE(y_train1, vthreshold_rating(reg_model.predict(X_train_wv_reg)))))
    print("Validation Error: {}".format(calculate_MSE(y_val1, vthreshold_rating(reg_model.predict(X_val_wv_reg)))))
    print()

Alpha = 0.0
------------


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=6.37186e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


Training Error: 0.8940535781682711
Validation Error: 0.9018064217232329

Alpha = 0.01
------------
Training Error: 0.9079188841981554
Validation Error: 0.9021185371572493

Alpha = 0.03
------------
Training Error: 0.907959198977622
Validation Error: 0.9021166181450053

Alpha = 0.1
------------
Training Error: 0.9080974389160185
Validation Error: 0.9021763444454539

Alpha = 0.3
------------
Training Error: 0.9083702951020889
Validation Error: 0.9023891975104213



In [89]:
learning_rates = [0.03, 0.1, 0.3, 0.5]
estimators = [50, 100, 200, 500, 1000, 2000]
depths = [1, 2, 5]

for learning_rate in learning_rates:
    for estimator in estimators:
        for depth in depths:
            print("Learning Rate: {0}, # Estimators: {1}, Depth: {2}".format(learning_rate, estimator, depth))
            print("--------------------------------------------------")
            xg_reg = XGBRegressor(
                learning_rate=learning_rate, max_depth=depth, n_estimators=estimator)
            xg_reg.fit(X_train_wv_reg, y_train1)
            print("Training Error: {}".format(calculate_MSE(y_train1, vthreshold_rating(xg_reg.predict(X_train_wv_reg)))))
            print("Validation Error: {}".format(calculate_MSE(y_val1, vthreshold_rating(xg_reg.predict(X_val_wv_reg)))))
            print()

Learning Rate: 0.03, # Estimators: 50, Depth: 1
--------------------------------------------------
Training Error: 1.6765857758092944
Validation Error: 1.6647381372164318

Learning Rate: 0.03, # Estimators: 50, Depth: 2
--------------------------------------------------
Training Error: 1.6580919193530188
Validation Error: 1.647719877424904

Learning Rate: 0.03, # Estimators: 50, Depth: 5
--------------------------------------------------
Training Error: 1.608621216796498
Validation Error: 1.6276771719437315

Learning Rate: 0.03, # Estimators: 100, Depth: 1
--------------------------------------------------
Training Error: 0.9686630666825585
Validation Error: 0.96021394548407

Learning Rate: 0.03, # Estimators: 100, Depth: 2
--------------------------------------------------
Training Error: 0.9469037643105891
Validation Error: 0.9420444818347719

Learning Rate: 0.03, # Estimators: 100, Depth: 5
--------------------------------------------------
Training Error: 0.8693182573628832
Validat

KeyboardInterrupt: 

### Meta Models

In this section we look at models that combine both collaborative filtering and language models.

We start by using the predictions of the collaborative filtering as features in our language model

In [104]:
columns_to_keep = ['cleanedPrice', 'isPop', 'isAlternativeRock', 'isJazz', 'isClassical', 'isDanceElectronic', 'reviewWordCount', 'userRating', 'itemRating']
X_train_reg2 = X_train_mod[columns_to_keep]
X_val_reg2 = X_val_mod[columns_to_keep]

In [105]:
min_max_scaler = MinMaxScaler()
X_train_reg2['reviewWordCount'] = X_train_reg2['reviewWordCount'].apply(lambda x: np.log(x))
X_val_reg2['reviewWordCount'] = X_val_reg2['reviewWordCount'].apply(lambda x: np.log(x))

<ipython-input-105-d8a296a7aa94>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_reg2['reviewWordCount'] = X_train_reg2['reviewWordCount'].apply(lambda x: np.log(x))
<ipython-input-105-d8a296a7aa94>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_reg2['reviewWordCount'] = X_val_reg2['reviewWordCount'].apply(lambda x: np.log(x))


In [106]:
X_train_reg2 = clean_dataset(X_train_reg2)
y_train2 = y_train[y_train.index.isin(X_train_reg2.index)]
X_train2 = X_train[X_train.index.isin(X_train_reg2.index)]

X_val_reg2 = clean_dataset(X_val_reg2)
y_val2 = y_val[y_val.index.isin(X_val_reg2.index)]
X_val2 = X_val[X_val.index.isin(X_val_reg2.index)]

<ipython-input-100-550e8da2fe0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-100-550e8da2fe0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [107]:
X_train_reg2 = min_max_scaler.fit_transform(X_train_reg2)
X_val_reg2 = min_max_scaler.transform(X_val_reg2)

In [108]:
alphas = [0.0, 0.01, 0.03, 0.1, 0.3]
for alpha in alphas:
    print("Alpha = {}".format(alpha))
    print("------------")
    reg_model = Ridge(alpha=alpha)
    reg_model.fit(X_train_reg2, y_train2)
    print("Training Error: {}".format(calculate_MSE(y_train2, vthreshold_rating(reg_model.predict(X_train_reg2)))))
    print("Validation Error: {}".format(calculate_MSE(y_val2, vthreshold_rating(reg_model.predict(X_val_reg2)))))
    print()

Alpha = 0.0
------------
Training Error: 0.7891828923294948
Validation Error: 0.9486008427597297

Alpha = 0.01
------------
Training Error: 0.7891677848703272
Validation Error: 0.9484601751838659

Alpha = 0.03
------------
Training Error: 0.7891678465440518
Validation Error: 0.9484577298907713

Alpha = 0.1
------------
Training Error: 0.7891680642355218
Validation Error: 0.948449171698209

Alpha = 0.3
------------
Training Error: 0.7891687018526102
Validation Error: 0.9484247228291963



In [109]:
exclude_english = set(stopwords.words('english'))
ps = PorterStemmer()
X_train2['processedReview'] = X_train2['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))
X_val2['processedReview'] = X_val2['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))

<ipython-input-109-25810bf4f6a4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train2['processedReview'] = X_train2['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))
<ipython-input-109-25810bf4f6a4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val2['processedReview'] = X_val2['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))


In [110]:
cv = CountVectorizer(max_features=1500)
X_train_cv2 = cv.fit_transform(X_train2['processedReview'])
X_val_cv2 = cv.transform(X_val2['processedReview'])

In [111]:
X_train_reg2_sp = sp.csr_matrix(X_train_reg2)
X_train_cv_reg2 = sp.hstack((X_train_cv2, X_train_reg2_sp), format='csr')

X_val_reg2_sp = sp.csr_matrix(X_val_reg2)
X_val_cv_reg2 = sp.hstack((X_val_cv2, X_val_reg2_sp), format='csr')

In [112]:
learning_rates = [0.01, 0.03, 0.1, 0.3, 0.5]
estimators = [50, 100, 200, 500]
depths = [1, 2, 5, 10]

for learning_rate in learning_rates:
    for estimator in estimators:
        for depth in depths:
            print("Learning Rate: {0}, # Estimators: {1}, Depth: {2}".format(learning_rate, estimator, depth))
            print("--------------------------------------------------")
            xg_reg = XGBRegressor(
                learning_rate=learning_rate, max_depth=depth, n_estimators=estimator)
            xg_reg.fit(X_train_cv_reg2, y_train2)
            print("Training Error: {}".format(calculate_MSE(y_train2, vthreshold_rating(xg_reg.predict(X_train_cv_reg2)))))
            print("Validation Error: {}".format(calculate_MSE(y_val2, vthreshold_rating(xg_reg.predict(X_val_cv_reg2)))))
            print()

Learning Rate: 0.01, # Estimators: 50, Depth: 1
--------------------------------------------------
Training Error: 6.503735872991274
Validation Error: 6.883393063136512

Learning Rate: 0.01, # Estimators: 50, Depth: 2
--------------------------------------------------
Training Error: 6.472925759109321
Validation Error: 6.873810349242472

Learning Rate: 0.01, # Estimators: 50, Depth: 5
--------------------------------------------------
Training Error: 6.415552951975483
Validation Error: 6.766778004848318

Learning Rate: 0.01, # Estimators: 50, Depth: 10
--------------------------------------------------
Training Error: 6.336512333143151
Validation Error: 6.702224040577343

Learning Rate: 0.01, # Estimators: 100, Depth: 1
--------------------------------------------------
Training Error: 2.9116597565047635
Validation Error: 3.2502935567689333

Learning Rate: 0.01, # Estimators: 100, Depth: 2
--------------------------------------------------
Training Error: 2.868043939305317
Validation E

Training Error: 0.06626693654491513
Validation Error: 0.629127338706747

Learning Rate: 0.3, # Estimators: 50, Depth: 1
--------------------------------------------------
Training Error: 0.6695470399035651
Validation Error: 0.8320175368867865

Learning Rate: 0.3, # Estimators: 50, Depth: 2
--------------------------------------------------
Training Error: 0.5877134032041788
Validation Error: 0.7312280170602444

Learning Rate: 0.3, # Estimators: 50, Depth: 5
--------------------------------------------------
Training Error: 0.821128725428028
Validation Error: 0.6644344297462841

Learning Rate: 0.3, # Estimators: 50, Depth: 10
--------------------------------------------------
Training Error: 0.18017706776835557
Validation Error: 0.6767750787866058

Learning Rate: 0.3, # Estimators: 100, Depth: 1
--------------------------------------------------
Training Error: 0.6250522630858881
Validation Error: 0.7799276543267967

Learning Rate: 0.3, # Estimators: 100, Depth: 2
----------------------

It seems `learning_rate=0.3`, `n_estimators=200`, `max_depth=2` performs the best

In [113]:
xg_reg = XGBRegressor(learning_rate=0.3, n_estimators=200, max_depth=2)
xg_reg.fit(X_train_cv_reg2, y_train2)

train_MSE = calculate_MSE(y_train2, vthreshold_rating(xg_reg.predict(X_train_cv_reg2)))
val_MSE = calculate_MSE(y_val2, vthreshold_rating(xg_reg.predict(X_val_cv_reg2)))

print("Training error based on XGBoost CountVectorizer prediction: %.3f" % train_MSE)
print("Validation error based on XGBoost CountVectorizer prediction: %.3f" % val_MSE)

Training error based on XGBoost CountVectorizer prediction: 0.502
Validation error based on XGBoost CountVectorizer prediction: 0.652


This is actually much worse compared to the pure language model.

However, we could also create a meta model by taking a weighted average of predictions from collaborative filtering and the pure language model. We now try this for a few candidate weightings

In [114]:
weights = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]

xg_reg = XGBRegressor(learning_rate=0.3, n_estimators=500, max_depth=2)
xg_reg.fit(X_train_cv_reg1, y_train1)

reg_train_preds = vthreshold_rating(xg_reg.predict(X_train_cv_reg1))
reg_val_preds = vthreshold_rating(xg_reg.predict(X_val_cv_reg1))

cf_train_preds = vthreshold_rating(X_train_mod['pred'])
cf_val_preds = vthreshold_rating(X_val_mod['pred'])

for weight in weights:
    print("Weight: %.1f" % weight)
    print("------------")
    train_MSE = calculate_MSE(y_train1, ((weight*reg_train_preds) + ((1.0 - weight)*cf_train_preds)))
    val_MSE = calculate_MSE(y_val1, ((weight*reg_val_preds) + ((1.0 - weight)*cf_val_preds)))
    print("Training error: %.3f" % train_MSE)
    print("Validation error: %.3f" % val_MSE)
    print()

Weight: 0.0
------------
Training error: 0.812
Validation error: 0.950

Weight: 0.1
------------
Training error: 0.754
Validation error: 0.882

Weight: 0.3
------------
Training error: 0.658
Validation error: 0.768

Weight: 0.5
------------
Training error: 0.587
Validation error: 0.681

Weight: 0.7
------------
Training error: 0.542
Validation error: 0.621

Weight: 0.9
------------
Training error: 0.522
Validation error: 0.589

Weight: 1.0
------------
Training error: 0.522
Validation error: 0.583



### Language and Popularity

Combining a language model with features representing how popular an item is (how many times it was rated) and how often the user rates.

In [38]:
user_rating_counts = X_train.groupby(X_train['reviewerID'])['price'].count()
item_rating_counts = X_train.groupby(X_train['itemID'])['price'].count()
user_rating_counts = pd.DataFrame(
    user_rating_counts.values, columns=['userRatingCount'], index=user_rating_counts.index).reset_index()
item_rating_counts = pd.DataFrame(
    item_rating_counts.values, columns=['itemRatingCount'], index=item_rating_counts.index).reset_index()

In [75]:
def null_to_zero(val):
    """Converts `val` to 0 if it is null. Otherwise, `val` is unchanged.
    
    Parameters
    ----------
    val: int
        The value to be converted.
    
    Returns
    -------
    float
        0 if `val` is null. Otherwise `val`.
    
    """
    if pd.isnull(val):
        return 0
    return val

def append_user_item_counts(data_df, user_counts, item_counts):
    """Appends `user_counts` and `item_counts` to `data_df`.
    
    Parameters
    ----------
    data_df: pd.DataFrame
        The DataFrame being appended to.
    user_counts: pd.DataFrame
        A DataFrame containing user counts.
    item_counts: pd.DataFrame
        A DataFrame containing item counts.
    
    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from `data_df` after appending
        `user_counts` and `item_counts`.
    
    """
    user_df = pd.merge(data_df, user_counts, how='left', left_on='reviewerID', right_on='reviewerID')
    user_item_df = pd.merge(user_df, item_counts, how='left', left_on='itemID', right_on='itemID')
    user_item_df['userRatingCount'] = user_item_df['userRatingCount'].apply(null_to_zero)
    user_item_df['itemRatingCount'] = user_item_df['itemRatingCount'].apply(null_to_zero)
    user_item_df.index = data_df.index
    return user_item_df

In [92]:
train_df = append_user_item_counts(X_train, user_rating_counts, item_rating_counts)
val_df = append_user_item_counts(X_val, user_rating_counts, item_rating_counts)

In [93]:
columns_to_keep = ['cleanedPrice', 'isPop', 'isAlternativeRock', 'isJazz', 'isClassical', 'isDanceElectronic', 'reviewWordCount', 'userRatingCount', 'itemRatingCount']
X_train_reg3 = train_df[columns_to_keep]
X_val_reg3 = val_df[columns_to_keep]

In [94]:
min_max_scaler = MinMaxScaler()
X_train_reg3['reviewWordCount'] = X_train_reg3['reviewWordCount'].apply(lambda x: np.log(x))
X_val_reg3['reviewWordCount'] = X_val_reg3['reviewWordCount'].apply(lambda x: np.log(x))

<ipython-input-94-063b9bc4b222>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_reg3['reviewWordCount'] = X_train_reg3['reviewWordCount'].apply(lambda x: np.log(x))
<ipython-input-94-063b9bc4b222>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_reg3['reviewWordCount'] = X_val_reg3['reviewWordCount'].apply(lambda x: np.log(x))


In [95]:
X_train_reg3 = clean_dataset(X_train_reg3)
y_train3 = y_train[y_train.index.isin(X_train_reg3.index)]
X_train3 = X_train[X_train.index.isin(X_train_reg3.index)]

X_val_reg3 = clean_dataset(X_val_reg3)
y_val3 = y_val[y_val.index.isin(X_val_reg3.index)]
X_val3 = X_val[X_val.index.isin(X_val_reg3.index)]

<ipython-input-47-ab573a02d313>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-47-ab573a02d313>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [96]:
X_train_reg3 = min_max_scaler.fit_transform(X_train_reg3)
X_val_reg3 = min_max_scaler.transform(X_val_reg3)

In [97]:
from sklearn.linear_model import Ridge

vthreshold_rating = np.vectorize(threshold_rating)

alphas = [0.0, 0.01, 0.03, 0.1, 0.3]
for alpha in alphas:
    print("Alpha = {}".format(alpha))
    print("------------")
    reg_model = Ridge(alpha=alpha)
    reg_model.fit(X_train_reg3, y_train3)
    print("Training Error: {}".format(calculate_MSE(y_train3, vthreshold_rating(reg_model.predict(X_train_reg3)))))
    print("Validation Error: {}".format(calculate_MSE(y_val3, vthreshold_rating(reg_model.predict(X_val_reg3)))))
    print()

Alpha = 0.0
------------
Training Error: 0.9568060307877613
Validation Error: 0.9646544613043789

Alpha = 0.01
------------
Training Error: 0.956800728863852
Validation Error: 0.9647191518211016

Alpha = 0.03
------------
Training Error: 0.9568009871918867
Validation Error: 0.9647195661109845

Alpha = 0.1
------------
Training Error: 0.9568018951045272
Validation Error: 0.9647210150463716

Alpha = 0.3
------------
Training Error: 0.9568045191684653
Validation Error: 0.9647251444048965



In [98]:
exclude_english = set(stopwords.words('english'))
ps = PorterStemmer()
X_train3['processedReview'] = X_train3['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))
X_val3['processedReview'] = X_val3['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))

<ipython-input-98-6f3f49189b95>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train3['processedReview'] = X_train3['fullReviewText'].apply(lambda x: process_review_text(x, exclude_english, ps))


In [99]:
cv = TfidfVectorizer()
X_train_cv3 = cv.fit_transform(X_train3['processedReview'])
X_val_cv3 = cv.transform(X_val3['processedReview'])

In [100]:
X_train_reg3_sp = sp.csr_matrix(X_train_reg3)
X_train_cv_reg3 = sp.hstack((X_train_cv3, X_train_reg3_sp), format='csr')

X_val_reg3_sp = sp.csr_matrix(X_val_reg3)
X_val_cv_reg3 = sp.hstack((X_val_cv3, X_val_reg3_sp), format='csr')

In [101]:
learning_rates = [0.03, 0.1, 0.3, 0.5]
estimators = [50, 100, 200, 500, 1000, 2000]
depths = [1, 2, 5]

for learning_rate in learning_rates:
    for estimator in estimators:
        for depth in depths:
            print("Learning Rate: {0}, # Estimators: {1}, Depth: {2}".format(learning_rate, estimator, depth))
            print("--------------------------------------------------")
            xg_reg = XGBRegressor(
                learning_rate=learning_rate, max_depth=depth, n_estimators=estimator)
            xg_reg.fit(X_train_cv_reg3, y_train3)
            print("Training Error: {}".format(calculate_MSE(y_train3, vthreshold_rating(xg_reg.predict(X_train_cv_reg3)))))
            print("Validation Error: {}".format(calculate_MSE(y_val3, vthreshold_rating(xg_reg.predict(X_val_cv_reg3)))))
            print()

Learning Rate: 0.03, # Estimators: 50, Depth: 1
--------------------------------------------------
Training Error: 1.6475736165735617
Validation Error: 1.6682075246391694

Learning Rate: 0.03, # Estimators: 50, Depth: 2
--------------------------------------------------
Training Error: 1.6071295532038965
Validation Error: 1.6310201443086647

Learning Rate: 0.03, # Estimators: 50, Depth: 5
--------------------------------------------------
Training Error: 1.5028318707489112
Validation Error: 1.5495761671615798

Learning Rate: 0.03, # Estimators: 100, Depth: 1
--------------------------------------------------
Training Error: 0.9188830540222396
Validation Error: 0.9353547910345448

Learning Rate: 0.03, # Estimators: 100, Depth: 2
--------------------------------------------------
Training Error: 0.8600495191978598
Validation Error: 0.881932049456795

Learning Rate: 0.03, # Estimators: 100, Depth: 5
--------------------------------------------------
Training Error: 0.7201781189903523
Vali

Training Error: 0.20162587407172136
Validation Error: 0.5881889589189081

Learning Rate: 0.3, # Estimators: 1000, Depth: 1
--------------------------------------------------
Training Error: 0.46988545890059963
Validation Error: 0.5847003633397732

Learning Rate: 0.3, # Estimators: 1000, Depth: 2
--------------------------------------------------
Training Error: 0.3362558793070724
Validation Error: 0.5794432857811497

Learning Rate: 0.3, # Estimators: 1000, Depth: 5
--------------------------------------------------
Training Error: 0.12375197406176025
Validation Error: 0.5985382001059424

Learning Rate: 0.3, # Estimators: 2000, Depth: 1
--------------------------------------------------
Training Error: 0.39958957988091076
Validation Error: 0.5768256291512879

Learning Rate: 0.3, # Estimators: 2000, Depth: 2
--------------------------------------------------
Training Error: 0.25034101761429917
Validation Error: 0.5806002474770103

Learning Rate: 0.3, # Estimators: 2000, Depth: 5
--------

This is not any better than the pure language model